In [1]:
import pandas as pd
import numpy as np
import glob
import os
from openpyxl import load_workbook
pd.options.display.float_format = '{:,.2f}'.format

In [2]:
start_date = '2022-07-30'
final_date = '2022-08-31'
query_error_year = 2022
query_error_month = 8

# INFORMACION DE  CUBETA ERROR

In [3]:
query_error = pd.read_csv(r'C:\Users\Cristian.Aparicio\Documents\POLIZAS\2022\08. AGOSTO\DATA\ERROR AGOSTO 2022.csv',
                          dtype={'report_id':object})
query_error.rename(columns ={'MAX(report_date_local)':'report_date_local'}, inplace=True)
query_error['report_date_local']= pd.to_datetime(query_error['report_date_local'])
query_error['ultimo_mov'] = pd.to_datetime(query_error['ultimo_mov'])
query_error['merchant_id_query'] = query_error['merchant_id']
query_error['ultimo_mov_date_only']= pd.to_datetime(query_error['ultimo_mov']).dt.date
query_error['day'] = query_error['ultimo_mov'].dt.day
query_error['month'] = query_error['ultimo_mov'].dt.month
query_error['year'] = query_error['ultimo_mov'].dt.year
query_error.loc[((query_error['day'] == 30 )
                & (query_error['month'] == 7)
                & (query_error['year'] == 2022)), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['day'] == 31 )
            & (query_error['month'] == 7 )
            & (query_error['year'] == 2022), "mes_cierre"] = 'Actual'
query_error.loc[(query_error['month'] == query_error_month )
            & (query_error['year'] == query_error_year), "mes_cierre"] = 'Actual'
query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
0,0,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,GTS92WH86,2015-11-05,734.34,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
1,1,b088c4b4-29dd-42c8-bd29-1eda61f73973,Mscelanea Dulce Maria,8VQRDV39G,2015-11-06,597.26,ERROR,swap devuelto,2017-08-15,8,2017,ERROR,ERROR_USUAL,b088c4b4-29dd-42c8-bd29-1eda61f73973,2017-08-15,15,NaN
2,2,d1087b80-5383-4b90-8738-bf309a1ce297,Grupo Solutec,82G5CYSR7,2015-12-21,"9,160.77",ERROR,failed swap,2017-11-01,11,2017,ERROR,ERROR_USUAL,d1087b80-5383-4b90-8738-bf309a1ce297,2017-11-01,1,NaN
3,3,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,XWRKLC2ZL,2016-02-08,"1,216.98",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN
4,4,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,Ediciones Malinalco S de RL de CV,D9468C7W4,2016-02-16,"1,993.14",ERROR,confirmar la cuenta,2017-05-12,5,2017,ERROR,ERROR_USUAL,8f666f7d-5eb8-4367-9c52-5564d6aca0c5,2017-05-12,12,NaN


In [4]:
query_error.shape

(100114, 17)

In [5]:
query_error.dtypes

Unnamed: 0                       int64
merchant_id                     object
merchant_name                   object
report_id                       object
report_date_local       datetime64[ns]
amount                         float64
status                          object
error_description               object
ultimo_mov              datetime64[ns]
month                            int64
year                             int64
status_cubeta                   object
esquema_query                   object
merchant_id_query               object
ultimo_mov_date_only            object
day                              int64
mes_cierre                      object
dtype: object

In [6]:
query_error['esquema_query'].unique()

array(['ERROR_USUAL', 'SIN_CLABE'], dtype=object)

In [7]:
query_error['status_cubeta'].unique()

array(['ERROR', 'NEW'], dtype=object)

# FILTRAR INFORMACION DE ACUERDO AL MES EN CURSO 

In [8]:
query_error_current_month = query_error[(query_error['mes_cierre'] == 'Actual')]
query_error_current_month.sort_values(by=['day'],inplace=True)
query_error_current_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10796\3502651561.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_error_current_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre
87000,87000,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,Sabe Dulce Cancún,TW3T3RNZS,2022-07-31,378.50,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,2022-08-01,1,Actual
87189,87189,ffa8c218-0c5a-4772-b67b-910b5a344ca9,Lima limon,JR7NMNYLZ,2022-07-31,"7,242.38",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-08-01,8,2022,NEW,ERROR_USUAL,ffa8c218-0c5a-4772-b67b-910b5a344ca9,2022-08-01,1,Actual
87190,87190,d412e33a-230b-4f6f-b89e-d25c7e6455d5,Sagitario,87TVP3T66,2022-07-31,109.72,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,d412e33a-230b-4f6f-b89e-d25c7e6455d5,2022-08-01,1,Actual
87191,87191,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,Paletería La Michoacana,526KJJNVB,2022-07-31,144.69,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,2022-08-01,1,Actual
87192,87192,835868c9-7ee3-4f4f-9366-79f8a58448e3,Tecate Six Sustaita,YLY2CCFFC,2022-07-31,"2,029.41",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,835868c9-7ee3-4f4f-9366-79f8a58448e3,2022-08-01,1,Actual


In [9]:
query_error_current_month.shape

(14424, 17)

In [10]:
query_error_current_month['month'].unique()

array([8, 7], dtype=int64)

In [11]:
query_error_current_month.columns

Index(['Unnamed: 0', 'merchant_id', 'merchant_name', 'report_id',
       'report_date_local', 'amount', 'status', 'error_description',
       'ultimo_mov', 'month', 'year', 'status_cubeta', 'esquema_query',
       'merchant_id_query', 'ultimo_mov_date_only', 'day', 'mes_cierre'],
      dtype='object')

In [12]:
duplicates_query_error = query_error_current_month[query_error_current_month.duplicated(['report_id'] , keep=False)]
duplicates_query_error.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [13]:
duplicates_query_error.shape

(0, 17)

# PAGOS NEW DE MESES ANTERIORES

In [14]:
query_new_last_month = query_error[(query_error['mes_cierre'] != 'Actual') & (query_error['status_cubeta'] == 'NEW')]
query_new_last_month.sort_values(by=['day'],inplace=True)
query_new_last_month.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10796\3381233170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  query_new_last_month.sort_values(by=['day'],inplace=True)


,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre


In [15]:
query_new_last_month.shape

(0, 17)

# INFORMACIÓN QUERIES - PAGOS

In [16]:
name_header = ['merchant_id','merchant_name','user_email','report_id','report_date_local','amount','status','bank_name'
               ,'payment_date_local','bank_account_owner_name','bank_account_number','swap']

### NEXT MONTH

In [17]:
# TRADICIONAL
# trad_files_csv_2021_may = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
# fp_files_csv_2021_may = glob.glob(r'C:\Users\Cristian.Aparicio\Documents\FAST PAYMENTS\2021\06. JUNE\QUERIES\*.csv', recursive=False)

In [18]:
# disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_may])
# disp_files_csv_2021_may['date_query'] = disp_files_csv_2021_may["file_name"].str[-14:-4]
# disp_files_csv_2021_may['date_query']= pd.to_datetime(disp_files_csv_2021_may['date_query'])

In [19]:
# fp_disp_files_csv_2021_may = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
#                             .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_may])
# fp_disp_files_csv_2021_may['date_query'] = fp_disp_files_csv_2021_may["file_name"].str[-14:-4]
# fp_disp_files_csv_2021_may['date_query']= pd.to_datetime(fp_disp_files_csv_2021_may['date_query']) 

### CURRENT MONTH

In [20]:
# TRADICIONAL
trad_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\08. AUGUST\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2022_current = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\08. AGOSTO\QUERIES\*.csv', recursive=False)

In [21]:
disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2022_current])
disp_files_csv_2022_current['date_query'] = disp_files_csv_2022_current["file_name"].str[-14:-4]
disp_files_csv_2022_current['date_query']= pd.to_datetime(disp_files_csv_2022_current['date_query'])

In [22]:
fp_disp_files_csv_2022_current = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2022_current])
fp_disp_files_csv_2022_current['date_query'] = fp_disp_files_csv_2022_current["file_name"].str[-14:-4]
fp_disp_files_csv_2022_current['date_query']= pd.to_datetime(fp_disp_files_csv_2022_current['date_query']) 

### LAST MONTH

In [23]:
# TRADICIONAL
trad_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\07. JULY\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_last = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\07. JULIO\QUERIES\*.csv', recursive=False)

In [24]:
disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_last])
disp_files_csv_2021_last['date_query'] = disp_files_csv_2021_last["file_name"].str[-14:-4]
disp_files_csv_2021_last['date_query']= pd.to_datetime(disp_files_csv_2021_last['date_query'])

In [25]:
fp_disp_files_csv_2021_last = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_last])
fp_disp_files_csv_2021_last['date_query'] = fp_disp_files_csv_2021_last["file_name"].str[-14:-4]
fp_disp_files_csv_2021_last['date_query']= pd.to_datetime(fp_disp_files_csv_2021_last['date_query']) 

### SPECIAL MONTH

In [26]:
# TRADICIONAL
trad_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\01. TRADICIONAL\2022\06. JUNE\QUERIES\*.csv', recursive=False)
# trad_files_txt_2021_apr = glob.glob(r'G:\My Drive\Settlement\5.-HISTORICO FILES PAGOS\2021\04. APRIL\QUERIES\*.txt', recursive=False)
# FAST PAYMENTS
fp_files_csv_2021_spec = glob.glob(r'G:\.shortcut-targets-by-id\1Oe28Mf9uSidMamjllDen2gkNnC59eSUr\3.-Settlement\5.-HISTORICO FILES PAGOS\02. FAST PAYMENTS\2022\06. JUNIO\QUERIES\*.csv', recursive=False)

In [27]:
disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in trad_files_csv_2021_spec])
disp_files_csv_2021_spec['date_query'] = disp_files_csv_2021_spec["file_name"].str[-14:-4]
disp_files_csv_2021_spec['date_query']= pd.to_datetime(disp_files_csv_2021_spec['date_query'])

In [28]:
fp_disp_files_csv_2021_spec = pd.concat([pd.read_csv(f,dtype={'bank_account_number':object,'report_id':object})
                            .assign(file_name=os.path.basename(f)) for f in fp_files_csv_2021_spec])
fp_disp_files_csv_2021_spec['date_query'] = fp_disp_files_csv_2021_spec["file_name"].str[-14:-4]
fp_disp_files_csv_2021_spec['date_query']= pd.to_datetime(fp_disp_files_csv_2021_spec['date_query']) 

## ALL FILES

In [29]:
allfiles = pd.concat([disp_files_csv_2022_current,fp_disp_files_csv_2022_current
                     ,disp_files_csv_2021_last,fp_disp_files_csv_2021_last,
                     disp_files_csv_2021_spec,fp_disp_files_csv_2021_spec])
allfiles.reset_index()
allfiles.fillna('BLANK',inplace=True)
allfiles['date_query']= pd.to_datetime(allfiles['date_query'])
allfiles['report_date_local']= pd.to_datetime(allfiles['report_date_local'])
allfiles.loc[(allfiles['file_name'].str.contains('FP'),'tipo_dispersion')] = 'FP'
allfiles.loc[(allfiles['file_name'].str.contains('SWAP'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('BNTE'),'tipo_dispersion')] = 'TRAD'
allfiles.loc[(allfiles['file_name'].str.contains('SANT'),'tipo_dispersion')] = 'TRAD'
allfiles.sort_values(by=['date_query','report_id'],inplace=True)
allfiles.drop(['status'],axis=1,inplace=True)
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion
28321,BLANK,5900e2ae-95c0-4f20-854d-bb600e796611,Abarrotes Martinez,martinez2614200@gmail.com,222482STQ,2022-06-01,222.31,"BBVA Bancomer, S.A.",2022-06-01 10:39:08,Yareli Martinez,'012078015846643487,2022-06-01 08:07:55,SANTANDER,"Yareli Martinez,,222.31,012078015846643487,CLI...",SANT 2022-06-01.csv,2022-06-01,TRAD
52937,BLANK,a5b45977-79fa-42e5-9f5f-ec69c98ae0a8,Vet Park,fonscastillo20@gmail.com,222JDFDJ2,2022-06-01,316.22,"Banco Regional de Monterrey, S.A.",2022-06-01 08:16:48,Alfonso Castillo Paredes,'058597000024045276,2022-06-01 08:11:17,SANTANDER,"Alfonso Castillo Paredes,,316.22,0585970000240...",SANT 2022-06-01.csv,2022-06-01,TRAD
3273,BLANK,0a0da737-0ca9-456d-b4f2-24542872bebb,Dimex Capital SA de CV SOFOM ENR,dimexcentro2clip@gmail.com,222N8GP7X,2022-06-01,"39,239.95","Banco Mercantil del Norte, S.A.",2022-06-01 10:37:23,Dimex Capital SA DE CV SOFOM ENR,'072580006215798028,2022-06-01 08:03:59,SANTANDER,"Dimex Capital SA DE CV SOFOM ENR,,39239.95,072...",SANT 2022-06-01.csv,2022-06-01,TRAD
18976,BLANK,3b722e3d-4889-46e8-bf82-9dd1c8246dc2,Burger Boxing,Victorburrola0@gmail.com,223QQNFT3,2022-06-01,"1,626.71","BBVA Bancomer, S.A.",2022-06-01 08:16:48,Victor Burrola Ramirez,'012180015431432085,2022-06-01 08:06:36,SANTANDER,"Victor Burrola Ramirez ,,1626.71,0121800154314...",SANT 2022-06-01.csv,2022-06-01,TRAD
16781,BLANK,347cce50-fd3e-4ec0-a5e8-c5b1799c2b72,Autoservicio Lulu,blourdesrc@hotmail.com,2244LHVBP,2022-06-01,378.98,"Banco Azteca, S.A.",2022-06-01 08:15:14,JOSE LUIS ESTRADA LOPEZ,'127320013386631426,2022-06-01 08:06:17,SANTANDER,"JOSE LUIS ESTRADA LOPEZ,,378.98,12732001338663...",SANT 2022-06-01.csv,2022-06-01,TRAD


In [30]:
def tipo_pago(allfiles):
    if allfiles['date_query'] == allfiles['report_date_local']:
        return 'Pago del día'
    else:
        return 'Reprogramación'
allfiles['tipo_pago'] = allfiles.apply(tipo_pago, axis=1)

In [31]:
allfiles.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount,bank_name,payment_date_local,bank_account_owner_name,bank_account_number,batch_created_date,disbursement_method,swap,file_name,date_query,tipo_dispersion,tipo_pago
28321,BLANK,5900e2ae-95c0-4f20-854d-bb600e796611,Abarrotes Martinez,martinez2614200@gmail.com,222482STQ,2022-06-01,222.31,"BBVA Bancomer, S.A.",2022-06-01 10:39:08,Yareli Martinez,'012078015846643487,2022-06-01 08:07:55,SANTANDER,"Yareli Martinez,,222.31,012078015846643487,CLI...",SANT 2022-06-01.csv,2022-06-01,TRAD,Pago del día
52937,BLANK,a5b45977-79fa-42e5-9f5f-ec69c98ae0a8,Vet Park,fonscastillo20@gmail.com,222JDFDJ2,2022-06-01,316.22,"Banco Regional de Monterrey, S.A.",2022-06-01 08:16:48,Alfonso Castillo Paredes,'058597000024045276,2022-06-01 08:11:17,SANTANDER,"Alfonso Castillo Paredes,,316.22,0585970000240...",SANT 2022-06-01.csv,2022-06-01,TRAD,Pago del día
3273,BLANK,0a0da737-0ca9-456d-b4f2-24542872bebb,Dimex Capital SA de CV SOFOM ENR,dimexcentro2clip@gmail.com,222N8GP7X,2022-06-01,"39,239.95","Banco Mercantil del Norte, S.A.",2022-06-01 10:37:23,Dimex Capital SA DE CV SOFOM ENR,'072580006215798028,2022-06-01 08:03:59,SANTANDER,"Dimex Capital SA DE CV SOFOM ENR,,39239.95,072...",SANT 2022-06-01.csv,2022-06-01,TRAD,Pago del día
18976,BLANK,3b722e3d-4889-46e8-bf82-9dd1c8246dc2,Burger Boxing,Victorburrola0@gmail.com,223QQNFT3,2022-06-01,"1,626.71","BBVA Bancomer, S.A.",2022-06-01 08:16:48,Victor Burrola Ramirez,'012180015431432085,2022-06-01 08:06:36,SANTANDER,"Victor Burrola Ramirez ,,1626.71,0121800154314...",SANT 2022-06-01.csv,2022-06-01,TRAD,Pago del día
16781,BLANK,347cce50-fd3e-4ec0-a5e8-c5b1799c2b72,Autoservicio Lulu,blourdesrc@hotmail.com,2244LHVBP,2022-06-01,378.98,"Banco Azteca, S.A.",2022-06-01 08:15:14,JOSE LUIS ESTRADA LOPEZ,'127320013386631426,2022-06-01 08:06:17,SANTANDER,"JOSE LUIS ESTRADA LOPEZ,,378.98,12732001338663...",SANT 2022-06-01.csv,2022-06-01,TRAD,Pago del día


In [32]:
allfiles.shape

(8219178, 18)

# CRUCE 1 - MES ACTUAL

In [33]:
cruce_1 = allfiles.merge(query_error_current_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status'
                                                   ,'error_description','esquema_query','status_cubeta']],on='report_id',how='left')
cruce_1['month'].fillna('Not found',inplace=True)
cruce_1.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,BLANK,5900e2ae-95c0-4f20-854d-bb600e796611,Abarrotes Martinez,martinez2614200@gmail.com,222482STQ,2022-06-01,222.31,"BBVA Bancomer, S.A.",2022-06-01 10:39:08,Yareli Martinez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,BLANK,a5b45977-79fa-42e5-9f5f-ec69c98ae0a8,Vet Park,fonscastillo20@gmail.com,222JDFDJ2,2022-06-01,316.22,"Banco Regional de Monterrey, S.A.",2022-06-01 08:16:48,Alfonso Castillo Paredes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,BLANK,0a0da737-0ca9-456d-b4f2-24542872bebb,Dimex Capital SA de CV SOFOM ENR,dimexcentro2clip@gmail.com,222N8GP7X,2022-06-01,"39,239.95","Banco Mercantil del Norte, S.A.",2022-06-01 10:37:23,Dimex Capital SA DE CV SOFOM ENR,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,BLANK,3b722e3d-4889-46e8-bf82-9dd1c8246dc2,Burger Boxing,Victorburrola0@gmail.com,223QQNFT3,2022-06-01,"1,626.71","BBVA Bancomer, S.A.",2022-06-01 08:16:48,Victor Burrola Ramirez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,BLANK,347cce50-fd3e-4ec0-a5e8-c5b1799c2b72,Autoservicio Lulu,blourdesrc@hotmail.com,2244LHVBP,2022-06-01,378.98,"Banco Azteca, S.A.",2022-06-01 08:15:14,JOSE LUIS ESTRADA LOPEZ,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [34]:
cruce_1.shape

(8219178, 26)

In [35]:
duplicates_cruce_1_previo = cruce_1[cruce_1.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_previo.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
576,BLANK,3b1085fb-300f-45d4-b73b-e3a29584dee6,Abarrotes Aguila,aguilalaura24@gmail.com,27K37N842,2022-06-01,"4,404.10","Banco Azteca, S.A.",2022-06-01 08:16:48,Susana Aguila,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
686,BLANK,a9824d90-02cf-4885-8436-213d16e78e7b,Carniceria leonor,miriamcarrera109@gmail.com,28JW7D6S6,2022-06-01,"1,713.25",AZTECA,2022-06-01 08:16:48,Míriam Hernández carrera,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1653,BLANK,a0f371b2-f655-4b83-9cb5-0322cff130af,Abarrotes Y Cremera TOLEDO,dagora79@gmail.com,2KMZY26ZK,2022-06-01,224.24,AZTECA,2022-06-01 10:40:00,Daniel Gonzlez Ramirez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1958,BLANK,d43ded03-fe73-4adf-ba82-14019c2fcac9,Acium,picaro.pacuaro@gmail.com,2NHMW32VD,2022-06-01,"1,961.52","BBVA Bancomer, S.A.",2022-06-01 10:38:56,ANDREA PRZYBYSLAWSKI GERHARD,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2342,BLANK,9c15c082-afdb-43ef-96e0-031e3ed62a27,Floreria Diana,pinkmery16@hotmail.com,2RSQGCHHL,2022-06-01,"16,481.72",AZTECA,2022-06-01 10:39:37,Rosa maria,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [36]:
duplicates_cruce_1_previo.shape

(194193, 26)

In [37]:
cruce_1_final = cruce_1[(cruce_1['month'] != 'Not found')]
cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_1_final.head()

C:\Users\Cristian.Aparicio\AppData\Local\Temp\ipykernel_10796\1938979547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cruce_1_final.drop_duplicates(subset='report_id',keep='last', inplace=True)


,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
3883277,INSTANT,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,MGFGNXS6D,2022-07-15,"1,790.95","Opciones Empresariales del Noreste, S.A. de C....",2022-07-15 14:21:49,Maria cruz Ferrer Martinez,...,FP,Pago del día,"1,790.95",2022-08-10,2022-08-10,8.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4166586,INSTANT,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,NHHLMM2KP,2022-07-18,141.82,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-18 10:01:19,Maria cruz Ferrer Martinez,...,FP,Pago del día,141.82,2022-08-10,2022-08-10,8.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4205943,INSTANT,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,6T2HVXCNF,2022-07-19,468.10,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-19 14:21:46,Maria cruz Ferrer Martinez,...,FP,Pago del día,468.10,2022-08-10,2022-08-10,8.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4218424,INSTANT,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,C3NLS2GRC,2022-07-19,287.47,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-19 16:24:54,Maria cruz Ferrer Martinez,...,FP,Pago del día,287.47,2022-08-10,2022-08-10,8.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW
4328770,INSTANT,89ece6be-8a40-4cd0-970d-97eefcb26dc7,MISC MATY,maricruzfer0202@gmail.com,LXB8TFHFS,2022-07-20,134.15,"Opciones Empresariales del Noreste, S.A. de C....",2022-07-20 14:12:23,Maria cruz Ferrer Martinez,...,FP,Pago del día,134.15,2022-08-10,2022-08-10,8.00,ERROR,Error request by Backoffice,ERROR_USUAL,NEW


In [38]:
cruce_1_final.shape

(7944, 26)

In [39]:
duplicates_cruce_1_final = cruce_1_final[cruce_1_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_1_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [40]:
duplicates_cruce_1_final.shape

(0, 26)

# CRUCE 2 MES ACTUAL

In [41]:
cruce_2 = query_error_current_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_2['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_2.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,87000,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,Sabe Dulce Cancún,TW3T3RNZS,2022-07-31,378.50,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,2022-08-01,1,Actual,TRAD,Pago del día
1,87189,ffa8c218-0c5a-4772-b67b-910b5a344ca9,Lima limon,JR7NMNYLZ,2022-07-31,"7,242.38",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-08-01,8,2022,NEW,ERROR_USUAL,ffa8c218-0c5a-4772-b67b-910b5a344ca9,2022-08-01,1,Actual,TRAD,Pago del día
2,87190,d412e33a-230b-4f6f-b89e-d25c7e6455d5,Sagitario,87TVP3T66,2022-07-31,109.72,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,d412e33a-230b-4f6f-b89e-d25c7e6455d5,2022-08-01,1,Actual,FP,Pago del día
3,87191,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,Paletería La Michoacana,526KJJNVB,2022-07-31,144.69,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,2022-08-01,1,Actual,FP,Pago del día
4,87192,835868c9-7ee3-4f4f-9366-79f8a58448e3,Tecate Six Sustaita,YLY2CCFFC,2022-07-31,"2,029.41",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,835868c9-7ee3-4f4f-9366-79f8a58448e3,2022-08-01,1,Actual,FP,Pago del día


In [42]:
cruce_2.shape

(16493, 19)

In [43]:
cruce_2_final = cruce_2[(cruce_2['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
0,87000,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,Sabe Dulce Cancún,TW3T3RNZS,2022-07-31,378.50,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,f1808afb-e6bb-4ede-b137-f03d7b4cbec2,2022-08-01,1,Actual,TRAD,Pago del día
1,87189,ffa8c218-0c5a-4772-b67b-910b5a344ca9,Lima limon,JR7NMNYLZ,2022-07-31,"7,242.38",ERROR,DEV.SPEICUENTA BLOQUEADA,2022-08-01,8,2022,NEW,ERROR_USUAL,ffa8c218-0c5a-4772-b67b-910b5a344ca9,2022-08-01,1,Actual,TRAD,Pago del día
2,87190,d412e33a-230b-4f6f-b89e-d25c7e6455d5,Sagitario,87TVP3T66,2022-07-31,109.72,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,d412e33a-230b-4f6f-b89e-d25c7e6455d5,2022-08-01,1,Actual,FP,Pago del día
3,87191,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,Paletería La Michoacana,526KJJNVB,2022-07-31,144.69,ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,ff92a0ba-70a4-4d57-b66f-3b363b2906ed,2022-08-01,1,Actual,FP,Pago del día
4,87192,835868c9-7ee3-4f4f-9366-79f8a58448e3,Tecate Six Sustaita,YLY2CCFFC,2022-07-31,"2,029.41",ERROR,DEV.SPEIEXCEDE EL LNMITE DE AB,2022-08-01,8,2022,NEW,ERROR_USUAL,835868c9-7ee3-4f4f-9366-79f8a58448e3,2022-08-01,1,Actual,FP,Pago del día


In [44]:
cruce_2_final.shape

(10013, 19)

In [45]:
duplicates_cruce_2_final = cruce_2_final[cruce_2_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_2_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_id,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,tipo_dispersion,tipo_pago
106,83804,8ec8c0f1-8202-456b-a0bc-3d020901b102,Deportes Rendon,6JMGL2VR2,2022-07-18,4.00,ERROR,DI,2022-08-01,8,2022,NEW,ERROR_USUAL,8ec8c0f1-8202-456b-a0bc-3d020901b102,2022-08-01,1,Actual,TRAD,Pago del día
107,83804,8ec8c0f1-8202-456b-a0bc-3d020901b102,Deportes Rendon,6JMGL2VR2,2022-07-18,4.00,ERROR,DI,2022-08-01,8,2022,NEW,ERROR_USUAL,8ec8c0f1-8202-456b-a0bc-3d020901b102,2022-08-01,1,Actual,TRAD,Reprogramación
108,83804,8ec8c0f1-8202-456b-a0bc-3d020901b102,Deportes Rendon,6JMGL2VR2,2022-07-18,4.00,ERROR,DI,2022-08-01,8,2022,NEW,ERROR_USUAL,8ec8c0f1-8202-456b-a0bc-3d020901b102,2022-08-01,1,Actual,TRAD,Reprogramación
109,84815,eb5bd47e-a37e-439d-a948-8acecb3c67f3,El tornillo,748JL8X34,2022-07-23,96.78,ERROR,DEV.SPEICUENTA EN OTRA DIVISA,2022-08-01,8,2022,NEW,ERROR_USUAL,eb5bd47e-a37e-439d-a948-8acecb3c67f3,2022-08-01,1,Actual,TRAD,Pago del día
110,84815,eb5bd47e-a37e-439d-a948-8acecb3c67f3,El tornillo,748JL8X34,2022-07-23,96.78,ERROR,DEV.SPEICUENTA EN OTRA DIVISA,2022-08-01,8,2022,NEW,ERROR_USUAL,eb5bd47e-a37e-439d-a948-8acecb3c67f3,2022-08-01,1,Actual,TRAD,Reprogramación


In [46]:
duplicates_cruce_2_final.shape

(3567, 19)

# AGRUPADO POR TIPO DE PAGO - MES ACTUAL

In [47]:
group_tipo_pago = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago.fillna(0,inplace=True)
group_tipo_pago.head()

ultimo_mov_date_only       amount_x                 \
esquema_query                           ERROR_USUAL                  
status_cubeta                                 ERROR                  
tipo_dispersion                                  FP           TRAD   
tipo_pago                            Reprogramación Reprogramación   
0                         2022-07-30           0.00           0.00   
1                         2022-07-31           0.00           0.00   
2                         2022-08-01         744.06         464.75   
3                         2022-08-02      20,773.71     289,246.53   
4                         2022-08-03           0.00       3,146.91   

                                                                         \
esquema_query                                                             
status_cubeta            NEW                                              
tipo_dispersion           FP                        TRAD                  
tipo_pago       Pago del día Reprogramación Pago del día Reprogramación   
0                   1,530.31           0.00   233,071.21         431.21   
1                  84,975.82           0.00   165,705.15          19.16   
2                 113,441.65           0.00   129,100.23         425.87   
3                   5,525.76           0.00    68,967.51     101,031.68   
4                       0.00           0.00    82,291.67      12,208.89   

                            
esquema_query          All  
status_cubeta               
tipo_dispersion             
tipo_pago                   
0               235,032.73  
1               250,700.13  
2               244,176.56  
3               485,545.19  
4                97,647.47

In [48]:
group_tipo_pago.shape

(34, 8)

In [49]:
group_esquema = pd.pivot_table(cruce_1_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema.fillna(0,inplace=True)
group_esquema.head()

ultimo_mov_date_only       amount_x                              \
esquema_query                         ERROR_USUAL                               
status_cubeta                               ERROR          NEW                  
tipo_pago                          Reprogramación Pago del día Reprogramación   
0                       2022-07-30           0.00   234,601.52         431.21   
1                       2022-07-31           0.00   250,680.97          19.16   
2                       2022-08-01       1,208.81   242,541.88         425.87   
3                       2022-08-02     310,020.24    74,493.27     101,031.68   
4                       2022-08-03       3,146.91    82,291.67      12,208.89   

                          
esquema_query        All  
status_cubeta             
tipo_pago                 
0             235,032.73  
1             250,700.13  
2             244,176.56  
3             485,545.19  
4              97,647.47

In [50]:
group_esquema.shape

(34, 5)

# CRUCE 3 - MES ANTERIOR

In [51]:
cruce_3 = allfiles.merge(query_new_last_month[['report_id','amount','ultimo_mov','ultimo_mov_date_only','month','status',
                                               'error_description','esquema_query','status_cubeta']],
                         on='report_id',how='left')
cruce_3['month'].fillna('Not found',inplace=True)
cruce_3.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta
0,BLANK,5900e2ae-95c0-4f20-854d-bb600e796611,Abarrotes Martinez,martinez2614200@gmail.com,222482STQ,2022-06-01,222.31,"BBVA Bancomer, S.A.",2022-06-01 10:39:08,Yareli Martinez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
1,BLANK,a5b45977-79fa-42e5-9f5f-ec69c98ae0a8,Vet Park,fonscastillo20@gmail.com,222JDFDJ2,2022-06-01,316.22,"Banco Regional de Monterrey, S.A.",2022-06-01 08:16:48,Alfonso Castillo Paredes,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
2,BLANK,0a0da737-0ca9-456d-b4f2-24542872bebb,Dimex Capital SA de CV SOFOM ENR,dimexcentro2clip@gmail.com,222N8GP7X,2022-06-01,"39,239.95","Banco Mercantil del Norte, S.A.",2022-06-01 10:37:23,Dimex Capital SA DE CV SOFOM ENR,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
3,BLANK,3b722e3d-4889-46e8-bf82-9dd1c8246dc2,Burger Boxing,Victorburrola0@gmail.com,223QQNFT3,2022-06-01,"1,626.71","BBVA Bancomer, S.A.",2022-06-01 08:16:48,Victor Burrola Ramirez,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN
4,BLANK,347cce50-fd3e-4ec0-a5e8-c5b1799c2b72,Autoservicio Lulu,blourdesrc@hotmail.com,2244LHVBP,2022-06-01,378.98,"Banco Azteca, S.A.",2022-06-01 08:15:14,JOSE LUIS ESTRADA LOPEZ,...,TRAD,Pago del día,NaN,NaT,NaN,Not found,NaN,NaN,NaN,NaN


In [52]:
cruce_3.shape

(8219178, 26)

In [53]:
cruce_3_final = cruce_3[(cruce_3['month'] != 'Not found')]
cruce_3_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [54]:
cruce_3_final.shape

(0, 26)

In [55]:
duplicates_cruce_3_final = cruce_3_final[cruce_3_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_3_final.head()

,type,merchant_id,merchant_name,user_email,report_id,report_date_local,amount_x,bank_name,payment_date_local,bank_account_owner_name,...,tipo_dispersion,tipo_pago,amount_y,ultimo_mov,ultimo_mov_date_only,month,status,error_description,esquema_query,status_cubeta


In [56]:
duplicates_cruce_3_final.shape

(0, 26)

# CRUCE 4 - MES ANTERIOR

In [57]:
cruce_4 = query_new_last_month.merge(allfiles[['report_id','tipo_dispersion','tipo_pago']],on='report_id',how='left')
cruce_4['tipo_dispersion'].fillna('Not found',inplace=True)
cruce_4.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [58]:
cruce_4.shape

(0, 19)

In [59]:
cruce_4_final = cruce_4[(cruce_4['tipo_dispersion'] != 'Not found')]
#cruce_2_final.drop_duplicates(subset='report_id',keep='last', inplace=True) 
cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [60]:
cruce_4_final.shape

(0, 19)

In [61]:
duplicates_cruce_4_final = cruce_4_final[cruce_4_final.duplicated(['report_id'] , keep=False)]
duplicates_cruce_4_final.head()

,Unnamed: 0,merchant_id,merchant_name,report_date_local,amount,status,error_description,ultimo_mov,month,year,status_cubeta,esquema_query,merchant_id_query,ultimo_mov_date_only,day,mes_cierre,report_id,tipo_dispersion,tipo_pago


In [62]:
duplicates_cruce_4_final.shape

(0, 19)

# AGRUPADO POR TIPO DE PAGO - MES ANTERIOR

In [63]:
group_tipo_pago_last_month = pd.pivot_table(cruce_3_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_dispersion',
                                                        'tipo_pago'],values=['amount_x']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_tipo_pago_last_month.fillna(0,inplace=True)
group_tipo_pago_last_month.head()

ValueError: No objects to concatenate

In [64]:
group_tipo_pago_last_month.shape

NameError: name 'group_tipo_pago_last_month' is not defined

In [65]:
group_esquema_last_month = pd.pivot_table(cruce_4_final,index=['ultimo_mov_date_only']
                                              ,columns=['esquema_query','status_cubeta','tipo_pago'],values=['amount']
                                              ,aggfunc=np.sum,margins=True).reset_index()
group_esquema_last_month.fillna(0,inplace=True)
group_esquema_last_month.head()

ValueError: No objects to concatenate

In [66]:
group_esquema_last_month.shape

NameError: name 'group_esquema_last_month' is not defined

In [67]:
writer = pd.ExcelWriter('FASE V. DATA POLIZA ERROR-USUAL AGOSTO 2022.xlsx',
                        engine='xlsxwriter',
                        engine_kwargs={'options': {'encoding':'utf-8'}})
group_tipo_pago.to_excel(writer, sheet_name='GRUPO TIPO PAGO')
group_esquema.to_excel(writer, sheet_name='GRUPO ESQUEMA')
# group_tipo_pago_last_month.to_excel(writer, sheet_name='GRUPO TIPO PAGO-MESES ANT')
# group_esquema_last_month.to_excel(writer, sheet_name='GRUPO ESQUEMA-MESES ANT')
cruce_1_final.to_excel(writer, sheet_name='DETALLE',index=False)
cruce_3_final.to_excel(writer, sheet_name='DETALLE-MESES ANTERIORES',index=False)
writer.close()